# Deploy huggingface's models

If you deploy models fron huggingface, just replace the model_id value.

These tested:

- Qwen2-7B-Instruct
- llama3
- chatglm3
- glm4-chat (bug: output will not stop even add stop_token_ids)
- phi3 (todo: clean response)


## Deploy

In [ ]:
import dotenv
import os
import sagemaker
import boto3

dotenv.load_dotenv(".env", override=True)

role_name = os.environ.get("role_name")

model_id="shenzhi-wang/Llama3-8B-Chinese-Chat" 
endpoint_name = "llama3"
model_id="THUDM/chatglm3-6b" 
endpoint_name = "glm3"
model_id="THUDM/glm-4-9b-chat" 
endpoint_name = "glm4-chat"
model_id="Qwen/Qwen2-7B-Instruct" 
endpoint_name = "qwen2"
model_id="microsoft/Phi-3-mini-128k-instruct" 
endpoint_name = "phi-3"

model_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
endpoint_name = "deepseek-15"

# model_id="llava-hf/llava-1.5-7b-hf"

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session._region_name


try:
    role = sagemaker.get_execution_role() # If you online sagemaker notebook
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName=role_name)["Role"]["Arn"]


image_uri = sagemaker.image_uris.retrieve(framework="djl-lmi", version="0.28.0", region=region)
instance_type = "ml.g5.2xlarge"

role_name, model_id, image_uri

In [ ]:
model = sagemaker.Model(
  image_uri=image_uri,
  role=role,
  env={
      "HF_MODEL_ID": model_id,
      # "OPTION_ROLLING_BATCH": "vllm",
      "HF_MODEL_TRUST_REMOTE_CODE": "True",
      # "GPU_MEMORY_UTILIZATION": "0.99"
  }
)

model.deploy(
  instance_type=instance_type, 
  initial_instance_count=1, 
  endpoint_name=endpoint_name
)

## Test

In [ ]:

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = sagemaker.Predictor(
  endpoint_name=endpoint_name,
  sagemaker_session=sagemaker_session,
  serializer=JSONSerializer(),
  deserializer=JSONDeserializer(),
)

In [ ]:

result = predictor.predict({
    "messages": [
      {
        "role": "system",
        "content": "You are a helpful assistant"
      },
      {
        "role": "user",
        "content": "who are you?"
      }
    ],
    "max_tokens":256
  })
print(result)

## Delete

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()